In [61]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
import numpy as np
from starsim.states import State, DynamicView
from removing_people import DynamicPeople

# Removing people concept

In [63]:
x = State('foo', int, 0)
z = State('bar', int, lambda n: np.random.randint(1,4,n))

p = DynamicPeople(states=[x,z])
p.initialize(3)


In [64]:
x

     Quantity
UID          
0           0
1           0
2           0

In [65]:
x.uid

array([0, 1, 2])

In [66]:
x.values

array([0, 0, 0])

In [67]:
z

     Quantity
UID          
0           2
1           2
2           2

In [68]:
p.grow(3)

In [69]:
x

     Quantity
UID          
0           0
1           0
2           0
3           0
4           0
5           0

In [70]:
z

     Quantity
UID          
0           2
1           2
2           2
3           1
4           3
5           3

## Some indexing operations

Different kinds of indexing

- Single index
- List/array of indices
- Boolean array
- Slices

In [71]:
x[2] = 10
x[3] = 20
x

     Quantity
UID          
0           0
1           0
2          10
3          20
4           0
5           0

In [72]:
x>0

     Quantity
UID          
0       False
1       False
2        True
3        True
4       False
5       False

In [73]:
x[x>0]

     Quantity
UID          
2          10
3          20

In [74]:
z

     Quantity
UID          
0           2
1           2
2           2
3           1
4           3
5           3

In [75]:
z+= 1

In [76]:
z

     Quantity
UID          
0           3
1           3
2           3
3           2
4           4
5           4

In [77]:
z[[1,3,5]] += 10

In [78]:
z

     Quantity
UID          
0           3
1          13
2           3
3          12
4           4
5          14

In [79]:
#np.mean(z) # Does not work: TypeError: FusedArray.mean() got an unexpected keyword argument 'axis'

# Instead use the following:
z.mean()

8.166666666666666

## Removing people

In [80]:
x

     Quantity
UID          
0           0
1           0
2          10
3          20
4           0
5           0

In [81]:
p.remove([0,1])

In [82]:
x

     Quantity
UID          
2          10
3          20
4           0
5           0

In [83]:
p.remove(2)

In [84]:
x

     Quantity
UID          
3          20
4           0
5           0

In [85]:
x[3]

20

In [86]:
try:
    x[0:3]
except Exception as e:
    print(e)

Slicing not supported - slice the .values attribute by index instead e.g., x.values[0:5], not x[0:5]


In [87]:
x[:]

     Quantity
UID          
3          20
4           0
5           0

In [88]:
x.values[0:3]

array([20,  0,  0])

## More assignment

In [89]:
x

     Quantity
UID          
3          20
4           0
5           0

In [90]:
x[[4,5]] = 1
x

     Quantity
UID          
3          20
4           1
5           1

In [91]:
z

     Quantity
UID          
3          12
4           4
5          14

In [92]:
x[:] = z
x

     Quantity
UID          
3          12
4           4
5          14

In [93]:
z[3] = 1
z

     Quantity
UID          
3           1
4           4
5          14

In [94]:
x

     Quantity
UID          
3          12
4           4
5          14

In [95]:
x[x<14] = 5
x

     Quantity
UID          
3           5
4           5
5          14

## cv.true

This example

```python
filter_inds = people.true('hiv')  # indices fo people with HIV
if len(filter_inds):    
    art_inds = filter_inds[hpu.true(people.art[filter_inds])]  # Indices of people on ART
    not_art_inds = filter_inds[hpu.false(people.art[filter_inds])]
    cd4_remaining_inds = hpu.itrue(((people.t - people.date_hiv[not_art_inds]) * dt) < people.dur_hiv[not_art_inds], not_art_inds)  # Indices of people not on ART who have an active infection
```

illustrates typical usage of the true() function - need to chain indexing i.e., subsequently index `filter_inds` rather than the original `People`.

In [96]:
p.grow(10)

In [97]:
z

     Quantity
UID          
3           1
4           4
5          14
6           1
7           2
8           2
9           2
10          1
11          1
12          2
13          1
14          2
15          2

In [98]:
z>1

     Quantity
UID          
3       False
4        True
5        True
6       False
7        True
8        True
9        True
10      False
11      False
12       True
13      False
14       True
15       True

In [99]:
def true(x):
    return x.uid.__array__()[np.nonzero(~x.__array__())]

In [100]:
gt_1 = true(z>1)
gt_1

array([ 3,  6, 10, 11, 13])

In [101]:
gt_1_lt_3 = true(z[gt_1]<3)
gt_1_lt_3

array([], dtype=int64)

Idea is we would be able to write things like

```python
hiv_uids = ss.true(hiv)
art_uids = ss.true(art[hiv_uids])
not_art_uids = ss.false(art[hiv_uids])
```

## Distributed states

- Define a state anywhere
- Connect it to a people instance at initialization

In [102]:
class Intervention():
    def __init__(self):
        self.vaccinated = State('day_vaccinated',bool)
    def initialize(self, people):
        # nb. in reality this will take in a Sim and use sim.people
        self.vaccinated.initialize(people)
        

In [103]:
my_intervention = Intervention()
my_intervention.vaccinated

<State day_vaccinated (uninitialized)>

In [104]:
len(p)

13

In [105]:
my_intervention.initialize(p)

In [106]:
my_intervention.vaccinated

     Quantity
UID          
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False

In [107]:
my_intervention.vaccinated.values[0:10] = True

In [108]:
my_intervention.vaccinated

     Quantity
UID          
3        True
4        True
5        True
6        True
7        True
8        True
9        True
10       True
11       True
12       True
13      False
14      False
15      False

In [109]:
p.grow(5)

In [110]:
my_intervention.vaccinated

     Quantity
UID          
3        True
4        True
5        True
6        True
7        True
8        True
9        True
10       True
11       True
12       True
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False

## Stateful modules

If the `State` object is itself stateful and the modules can be stateful too, then we don't need

```
class HIV(module):
    def __init__(self):
        self.states = [
            State('susceptible', bool, True),
            State('infected', bool, False),
            State('ti_infected', float, 0),
            State('ti_dead', float, np.nan),
            ]
```

instead we can actually just have

```
class HIV(module):
    def __init__(self):
        self.sus = State('susceptible', bool, True)
        self.infected = State('infected', bool, False)
        self.ti_infected = State('ti_infected', float, 0)
        self.ti_dead = State('ti_dead', float, np.nan)

```

Then these states can be initialized in-place, and referenced directly. So instead of


```
class HIV(module):
    def set_prognoses(self, sim, uids):
        sim.people[self.name].susceptible[uids] = False
        sim.people[self.name].infected[uids] = True
        sim.people[self.name].ti_infected[uids] = sim.ti
```

we could have

```
class HIV(module):
    def set_prognoses(self, sim, uids):
        self.susceptible[uids] = False
        self.infected[uids] = True
        self.ti_infected[uids] = sim.ti
```

That doesn't preclude ALSO having a reference to these same states in `people.hiv.susceptible`. This would mirror having `sim.pars.hiv` mirror `HIV.pars`

# Class structure

- UID map and UIDs need to be dynamic, but should be accessed by index rather than UID (`DynamicView`)
- States need to be dynamic and accessed by UID (`State(FusedArray)` containing a `DynamicView` where the `FusedArray` values reference the `DynamicView`'s `_view`)
- Indexing states need to return a container accessed by UID but with different UIDs to the people, and does not need to be dynamic (`FusedArray`)
